In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import os

import torch
import torch.nn as nn


In [2]:
#read in data
data_path = "/home/kishen/documents/python_projects/disaster_tweets/data/"
df = pd.read_csv(data_path + "clean_tweets.csv")

Preprocessing

In [3]:
os.chdir("/home/kishen/documents/python_projects/disaster_tweets/")
from src.processing import tokenizer

from sklearn.model_selection import train_test_split

#Split data
X, y = df['text'].values, df['target'].values



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= .2, shuffle=True, random_state=24)

#Sanity Check
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(8978,) (8978,)
(2245,) (2245,)


In [4]:
from src.datasets import Dataset, Vocabulary, MyCollate

train_loader = Dataset(X_train, y_train, freq_threshold=2)


In [5]:
from torch.utils.data import Dataset, DataLoader

# Parameters
loader_params = {'batch_size': 16,
          'shuffle': True,
          'num_workers': 8,
          'collate_fn':MyCollate(train_loader.vocab.stoi["<PAD>"])}
loader = DataLoader(dataset=train_loader, 
                    **loader_params)
# CUDA for PyTorch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
print(device)


cuda:0


In [6]:
from src.modeling import RNN
from torch.optim import SGD

model = RNN(input_size= loader_params['batch_size'], hidden_size= 64, output_size= 2)
optimizer_params = {'lr':.001, 
                    'momentum':.9}

criterion = nn.BCELoss()

optimizer = SGD(model.parameters(), **optimizer_params )

Training

In [17]:
epochs = 1

for epoch in range(epochs):
    for i, (data, targets) in enumerate(train_loader):
        #Move to GPU
        data = data.to(device=device).unsqueeze(-1)
        targets = targets.to(device=device)
        
        #Model computations
        preds = model(data)
        loss = criterion(preds, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    #validation
        



RuntimeError: input.size(-1) must be equal to input_size. Expected 16, got 1